### Displays house wiring data from Excel file in variable wiringSpreadsheetFNm

#### TODO
- [ ] need to denote "disconnect" from pre wiring
- [ ] need to explicitly say which outlets are dedicated
- [ ] tag group with AMPS, AFCI, GFCI

In [1]:
import pandas as pd
import numpy as np
from IPython.display import HTML
from icecream import ic

wiringSreadsheetFNm = 'house_rewiring_2023.xlsx'
#indexColNmsFvB = ['Breaker Pre', 'Room', 'Fixture ID']
indexColNmsFvB = ['Room', 'Fixture ID', 'Brkr Pre']
nBreakersInPanel = 24 # max number breakers possible

def display_tall(df: pd.DataFrame, no_index=False) -> None:
    """Displays dataframe, df, with no number-of-rows restriction"""
    orig_max_rows = pd.get_option('display.max_rows')
    pd.set_option('display.max_rows', None)
    
    if no_index:
        display(df.style.hide())
    else:
        display(df)

    pd.set_option('display.max_rows', orig_max_rows)

df_fix = pd.read_excel(wiringSreadsheetFNm, sheet_name='fixture v breaker') # 'breaker v cable'

#display_tall(df_fix.set_index(indexColNmsFvB).sort_index())

In [2]:
# sumColNms = ['Box OW int. 1G', 'Box OW ext 1G', 'Int. OW Box P 2G',
#        'Box OW Ext P 2G', 'Box NW M 2G', 'Box NW M 3G',
#        'Jbox P side mnt, round ', 'Jbox M (MC)', 'Box P OW ext.',
#        'Box ceiling OW P pnck', 'SW SPST', 'SW SPDT', 'Rcpt. 1G 20 A',
#        'GFCI 20A', 'Plate W S1 1G', 'Plate W  R2 1G', 'Plate W S2 2G',
#        'Plate M R2 2G', 'Plate M S2 2G', 'Plate M S0 2G', 'Plate M S3 3G',
#        'Plate M 1 sw, 4 recpt', 
#        '1 bulb fixture porc.' ]
# df_fix[sumColNms].sum().to_csv('~/tmp/fixtureBuy.csv')
# df_fix[sumColNms].sum()

In [3]:
display_tall(df_fix.reset_index()['Fixture ID'].sort_values())

45       F1
20     F10a
31       F2
34       F3
59       F4
78       F5
51       L1
30      L10
81      L10
84      L11
98      L12
10      L13
11     L13a
12      L14
13     L14a
14     L14b
15      L15
23      L16
24     L16b
16      L17
70      L17
71      L18
27      L18
28      L19
52       L2
53       L3
54       L4
55       L5
44       L6
36      L6a
66       L7
80       L8
57       L9
1        R1
19      R10
21      R11
22     R11a
43      R12
50      R13
49      R14
48      R15
47      R16
32     R16a
58      R17
61      R18
62     R18a
64      R19
2        R2
69      R20
67      R21
68     R21a
72      R22
73     R22a
74      R23
77     R23a
76      R24
37     R24a
38      R25
39     R25a
40      R26
92      R26
41     R26a
42     R26b
93      R27
94     R27a
95     R27b
96     R27c
97      R28
88      R29
89     R29a
90     R29b
91     R29c
3        R3
85      R30
86     R30a
87      R31
79      R32
4        R4
5        R5
7        R6
8        R7
9        R8
18       R9
0   

In [4]:
# Currrent fixtures on each braker
# for brkr, df_brkr in df_fix.reset_index().groupby(['group','Brkr Pre','Fixture ID']):
#     display(df_brkr)

#### Current Number and type of fixtures on each breaker

In [5]:
fixture_counts = pd.DataFrame()
for bk, df_bk in df_fix.groupby('Brkr Pre'):
    for t, fcnt in df_bk.Type.value_counts().items():
        fixture_counts.loc[bk, t] = fcnt
fixture_counts[fixture_counts.isna()] = 0
fixture_counts = fixture_counts.astype(int)
fixture_counts['Total'] = fixture_counts.sum(axis=1)

fixture_counts.columns.name = 'Fixture Type'
fixture_counts.index.name = 'Breaker'
column_order = ['R','L','S','F', 'Total']
fixture_counts = fixture_counts[column_order].sort_values(column_order, ascending=False)
fixture_counts.style.set_caption('Number and Type of Fixtures on Each Breaker')

Fixture Type,R,L,S,F,Total
Breaker,,,,,
0,17,8,2,3,36
"6,8",6,9,8,0,23
15,6,0,2,0,8
14,4,8,3,1,16
7,4,3,1,0,8
10,3,0,0,0,3
3,2,0,0,0,2
5,1,0,0,0,1
9,1,0,0,0,1


#### Pre *Cable* number vs. breaker

In [6]:
df_cab = pd.read_excel(wiringSreadsheetFNm, sheet_name='breaker v cable')
# Outgoing cable counts (avoids double breaker undercount, but could be overcount since some appear unconnected)
nCircuitsPre = len(df_cab) 
df_cab.set_index('Cable Num Brkr Pre')

,Brkr Pre,Amps Pre,Cable Type Brkr Pre,Destinations,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Comments
Cable Num Brkr Pre,,,,,,,,,,
11,9,20,rom1p,unconnected,NaN,NaN,NaN,NaN,NaN,NaN
15,15,20,rom1p,"clotheswasher R, garage RLS",NaN,NaN,NaN,NaN,NaN,NaN
13 black,13,20,rom2p,bathroom GFCI,NaN,NaN,NaN,NaN,NaN,NaN
13 red,13,20,rom2p,not connected,NaN,NaN,NaN,NaN,NaN,"unconn in box, wire nut on end"
5,11,20,arm1p,"R, ironing board",NaN,NaN,NaN,NaN,NaN,NaN
9,7,20,rom1p,"shop RLS, old phone R, patio R",NaN,NaN,NaN,NaN,NaN,NaN
7,5,20,rom1p,dishwasher,NaN,NaN,NaN,NaN,NaN,NaN
3,3,20,rom1p,NW wall kitchen GFCI,NaN,NaN,NaN,NaN,NaN,"bmnt light, upstairs trunk"
1,1,20,rom1p,garbage disposal (sw?),NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# I don't remember what this was for
# display_tall(df_cab.merge(df_fix.reset_index(), on='Brkr Pre', how='outer').set_index(indexColNmsFvB))

### Group Post Fixtures to Post Circuit Breakers

Possible groups:
- all lights in the dining room , living rooom, or exterior N wall
- all circuits where type is SM or SMCO
- receptical R10, R12
- receptical R1
- all lights in upstairs bedrooms
- all recpticals in upstairs room E
- all recepticlas in upstaris room W



##### Find pre rewiring breakers with single or few attached fixtures
Probably want them to be dedicated, like they were originally.  

**Decide if this is OK**

Things that won't be dedicated with this scheme are
- [ ] stove [probably OK](https://forums.mikeholt.com/threads/necs-definition-of-dedicated.16162/post-367174)
- [ ] garage outlets
- [ ] shop outlets

In [8]:
# TODO: washingmachine: R6 (force in small breaker)
# TODO: exclude small breaker fixtures from larger groups

brkrPreNm = 'Brkr Pre'
breaker_loads_pre = df_fix.groupby(brkrPreNm).count().iloc[:, 0]

# make a dataframe query() statement for breakers with few fixtures
queryForSmallBreaker = {}
for bkr in breaker_loads_pre[breaker_loads_pre < 5].index:
    df_info = df_fix.set_index(brkrPreNm).loc[bkr][['Fixture ID', 'Room', 'Details Pre']]
    if not isinstance(df_info, pd.DataFrame):
        df_info  = pd.DataFrame(df_info).T

    fixID_set_str = '[' + ','.join([f'"{item}"' for item in df_info['Fixture ID']]) + ']'
    queryForSmallBreaker[bkr] = f"`Fixture ID` in {fixID_set_str}"

#queryForSmallBreaker['2,4'] = f"`Fixture ID` in {fixID_set_str}"
display(queryForSmallBreaker)

# Assign group names and attributes for the small breakers found above.  
# For python >= 3.7, these dicts are ordered so top ones on this list are assigned 1st
# Fixed appliances: not GFCI'ed e.g. fridge, says EE phD, prof eng.: https://qr.ae/pKWcyM
# Exception: furnace on basement outlets, so both afci and gfci: 
# https://forums.mikeholt.com/threads/gfci-afci-for-furnace.2547011/post-2547042
groupInfoForSmallBreaker = {1:{'gname':'gdisposal', 'attr':['20A','AFCI']},
                            # local GFCI, so GFCI breaker not needed, but do I have to replace them?
                            3:{'gname':'kitchenGFCI_N', 'attr':['20A','AFCI']},
                            9:{'gname':'fridge', 'attr':['20A','AFCI']},
                            # includes old oven plug so fix later
                            10:{'gname':'kitchenGFCI_S', 'attr':['20A','AFCI']},
                            5:{'gname':'dishwasher', 'attr':['20A','AFCI']},
                            # the ironing board mega plugin
                            11:{'gname':'Laundry_R', 'attr':['20A','AFCI','GFCI']}, 
                            12:{'gname':'microwave', 'attr':['20A','AFCI']},
                            # local GFCI, so GFCI breaker not needed. Do I have to replace them?
                            13:{'gname':'BathroomGFCI', 'attr':['20A','AFCI']},
                            # furnace on basement outlets, so both afci and gfci: 
                            # https://forums.mikeholt.com/threads/gfci-afci-for-furnace.2547011/post-2547042
                            16:{'gname':'FurnaceWH', 'attr':['20A','AFCI', 'GFCI']}, 
                           '2,4':{'gname':'OldElecDryer', 'attr':['Removed']}}  # was unconnected

if (groupBrkrs := groupInfoForSmallBreaker.keys()) != (queryBrkrs := queryForSmallBreaker.keys()):
    groupBrkrs, queryBrkrs  = set(groupBrkrs), set(queryBrkrs)
    print(f"Breakers in groupInfo NOT in query: {groupBrkrs - queryBrkrs}")
    print(f"Breakers in queryInfo NOT in group: {queryBrkrs- groupBrkrs}")
    print(f'{groupBrks=}')
    print(f'{queryBrkrs=}')
    raise ValueError('small breaker key mismatch')

# assign in order of groupInfoForSmallBreaker
smallBreakerGroups = {info['gname']: {'query':queryForSmallBreaker[brkr], 'attr':info['attr']} 
                      for brkr, info in groupInfoForSmallBreaker.items()}

# Force some dedicated ckts not captured above. Put them on top of list so won't be overwritten

# I'm not sure Oven must be dedicated, but make it so.  Was it tripping the outlet GFCI? Dedicated appliance so no GFCI                         
forceBreakerDedicated = {'Oven': {'query':'`Fixture ID` in ["R14"]',  'attr':['20A', 'AFCI']},
                        # separate washer and dryer although probably OK on same ckt: https://www.electriciantalk.com/threads/oesc-gas-dryer-in-laundry-room-seperate-circuit-requirement.281148/post-5342152
                        #  I'll GFCI them b/c they're plugins
                         'clothesWasher': {'query':'`Fixture ID` in ["R6"]',  'attr':['20A', 'AFCI', 'GFCI']},
                         'clothesDryer': {'query':'`Fixture ID` in ["R10a"]',  'attr':['20A', 'AFCI', 'GFCI']}}
smallBreakerGroups = {**forceBreakerDedicated, **smallBreakerGroups}

smallBreakerGroups

{1: '`Fixture ID` in ["F4","S7"]',
 3: '`Fixture ID` in ["R18","R19"]',
 5: '`Fixture ID` in ["R18a"]',
 9: '`Fixture ID` in ["R17"]',
 10: '`Fixture ID` in ["R16","R14","R13"]',
 11: '`Fixture ID` in ["R8"]',
 12: '`Fixture ID` in ["R15"]',
 13: '`Fixture ID` in ["R12"]',
 16: '`Fixture ID` in ["R11"]',
 '2,4': '`Fixture ID` in ["R7"]'}

{'Oven': {'query': '`Fixture ID` in ["R14"]', 'attr': ['20A', 'AFCI']},
 'clothesWasher': {'query': '`Fixture ID` in ["R6"]',
  'attr': ['20A', 'AFCI', 'GFCI']},
 'clothesDryer': {'query': '`Fixture ID` in ["R10a"]',
  'attr': ['20A', 'AFCI', 'GFCI']},
 'gdisposal': {'query': '`Fixture ID` in ["F4","S7"]',
  'attr': ['20A', 'AFCI']},
 'kitchenGFCI_N': {'query': '`Fixture ID` in ["R18","R19"]',
  'attr': ['20A', 'AFCI']},
 'fridge': {'query': '`Fixture ID` in ["R17"]', 'attr': ['20A', 'AFCI']},
 'kitchenGFCI_S': {'query': '`Fixture ID` in ["R16","R14","R13"]',
  'attr': ['20A', 'AFCI']},
 'dishwasher': {'query': '`Fixture ID` in ["R18a"]', 'attr': ['20A', 'AFCI']},
 'Laundry_R': {'query': '`Fixture ID` in ["R8"]',
  'attr': ['20A', 'AFCI', 'GFCI']},
 'microwave': {'query': '`Fixture ID` in ["R15"]', 'attr': ['20A', 'AFCI']},
 'BathroomGFCI': {'query': '`Fixture ID` in ["R12"]', 'attr': ['20A', 'AFCI']},
 'FurnaceWH': {'query': '`Fixture ID` in ["R11"]',
  'attr': ['20A', 'AFCI', 'GFCI']

##### Define the big (non-dedicated) groups
TODO: Criteria is broad, so must prevent big groups from overwriting small groups, which are considered dedicated

was moved 9

In [17]:
# dedicated garage + exterior plugs
# 
# ordered dict meaans top on this list are assigned first (if not already assigned by small group)
bigBreakerGroups = {
    'smokedets': {'query':'Type in ["SMCO", "SM"]',  'attr':['15A', 'AFCI']},
    # Group of garage plugs must be dedicated but can be connected to outdoor receptacles
    # https://forums.mikeholt.com/threads/dedicated-garage-bay-plugs.2565919/post-2730329
    # exterior plugs don't have to be dedicated: https://qr.ae/pKWQF3
    # garage plugs have local GFCI already, as do outdoor plugs
    'garagExtSwall_GFCI': {'query':'(Room in ["Garage", "S Exterior Wall"]) and (Type in ["R"])', 'attr':['20A','AFCI']},
    # breaker GFCI in basement
    'Basement_R': {'query':'(Room in ["Basement"]) and (Type in ["R"])', 'attr':['20A','AFCI','GFCI']},
    'garagBsmnt_L': {'query':'(Room in ["Garage", "Basement", "Basement Stairs"]) and (Type in ["L", "S"])',  
                     'attr':['15A','AFCI']},
    # power comes from basement
    # need to exclude garbage disposal switch?
    # "F" is the bahtroom fan, that the garbage disposal
    # TODO: think of a better name
        'kitchenBath_LF': {'query':'((Room in ["Kitchen","E Exterior Wall", "Bathroom"]) and (Type in ["L", "S", "F"])) or (`Fixture ID` == "S5")',
                     'attr':['15A','AFCI']},
    # 'kitchDinRm_L': {'query':'(((Room in ["Kitchen","Dining Room"]) and (Type in ["L", "S", "F"])) or `Fixture ID` == "S5")  and ("Fixture ID" not in ["S7"])',
    #                  'attr':['15A','AFCI']},
    # get power from attic
    'officeDinRm_L': {'query':'(Room in ["Office", "Dining Room"]) and (Type in ["L", "S"])',
                          'attr':['15A','AFCI']},
    # upper floor lights, upper stairway also gets tpower from attic                          
    'upperBdrmsUpperStair_L': {'query':'(Room in ["Upper W Bdrm", "Upper E Bdrm", "Ground Hall", "Upper Hall"]) and (Type in ["L", "S"]) and ("Fixture ID" not in ["S5", "R16a"])',  
                               'attr':['15A','AFCI']},
    'upperFloor_R': {'query':'(Room in ["Upper W Bdrm",  "Upper E Bdrm", "Upper Hall"]) and (Type == "R")', 'attr':['20A','AFCI']},
    # ground floor R + patio & front step lights, doorbell (exterior lights each have a GFCI)
    'groundFloorR_extNS_LRF': {'query':'(Room in ["Living Room", "Dining Room", "Bathroom", "Office", "Ground Hall", "N Exterior Wall", "S Exterior Wall"]) and (Type in ["L", "R", "F", "S"])',  'attr':['20A','AFCI']},
    # This counts doorbell and alarm as "connections": a reminder, as button must be connected to low volts
    # TODO: Need a "disconnect" designator, and a 'maybe' designator
    }

allGroups = {**smallBreakerGroups, **bigBreakerGroups} # dict ordering is maintained
#allGroups

raise Exception("This still puts R16a (old hone R) and S5 (bathroom switches) in upperBdrmsUpperStair_L")
raise Exception("This still puts Front door switch in ground_LF")
raise Exception("Still puts garbage disposal on kitchen lights")


##### Merge w/ fixture info

In [18]:
df_fg = df_fix.copy()
unassignedNm = 'unassigned'
df_fg['group'] = unassignedNm

nFixtures = len(df_fg)


In [19]:
df_fg.query('`Fixture ID` == "R10a"')

,Fixture ID,Type,Room,Details Pre,New Details Post,Brkr Pre,Brkr Post,Amp Post,GFCI rqt,Box OW int. 1G,...,New 14 G (15 A) len.,1 bulb fixture porc.,Dim. SW,MC len.,MC conn.,SM and/or CO det,Salvage,Job,Questions,group


In [20]:
def assign_groups(df, groupspecs):
    for gname, groupInfo in groupspecs.items():
        try:
            qstr = f'{groupInfo["query"]} and (group == @unassignedNm)'
            assignIx = df.query(qstr).index
        except:
            raise ValueError(f"query error: {gname=}, {groupInfo}")
        #ic(gname, df.query('`Fixture ID` == "R10a"')['group'].values)

        df.loc[assignIx, 'group'] = gname
        # sort so unique combos are detect correctly, regardless of order in list
        df.loc[assignIx, 'attrStr'] = ", ".join(sorted(groupInfo['attr']))
        df.loc[assignIx, [f'is{a}' for a in groupInfo['attr']]] = True

    return df

df_fg = assign_groups(df_fg, allGroups) # assumes allGroups ordered so dedicated first
df_fg.fillna(False, inplace=True)

unassignedIx = df_fg.query('group == @unassignedNm').index
nBreakers = len(df_fg.group.unique())
if (nUnmatched := len(unassignedIx)) > 0:
    print(f'{nUnmatched} of {nFixtures} fixtures are unasigned using {nBreakers=}  (originally {nCircuitsPre=}):')
    display(df_fg.loc[unassignedIx,['Fixture ID', 'Room']])
else:
    print(f'All {nFixtures=} assigned using {nBreakers=} (originally {nCircuitsPre=})')

if nBreakers > (nBreakersNotSpare := nBreakersInPanel-2):
    raise ValueError(f'{nBreakers=} exceeds reserve, is greater than {nBreakersNotSpare}')

# set new fixture pre breaker ID to false (why weren't they already?)
df_fg.loc[df_fg['Brkr Pre'] == False, 'Brkr Pre'] = 0

All nFixtures=106 assigned using nBreakers=20 (originally nCircuitsPre=17)


In [21]:
for groupNm, df_this in df_fg.groupby('group'):
    display(df_this.set_index(['group','Brkr Pre', 'attrStr']))

Fixture ID Type      Room  \
group      Brkr Pre attrStr                                     
Basement_R 7        20A, AFCI, GFCI         R9    R  Basement   
                    20A, AFCI, GFCI        R10    R  Basement   
           0        20A, AFCI, GFCI       F10a    R  Basement   
                    20A, AFCI, GFCI       R11a    R  Basement   

                                              Details Pre  \
group      Brkr Pre attrStr                                 
Basement_R 7        20A, AFCI, GFCI  shop rcpt, 2 outlets   
                    20A, AFCI, GFCI   N wall behind shelf   
           0        20A, AFCI, GFCI                 False   
                    20A, AFCI, GFCI          W wall recpt   

                                                      New Details Post  \
group      Brkr Pre attrStr                                              
Basement_R 7        20A, AFCI, GFCI             now 4 outlets (2 gang)   
                    20A, AFCI, GFCI            moved, same box as R10a   
           0        20A, AFCI, GFCI  dedicated dryer, in 2G box w/ R10   
                    20A, AFCI, GFCI                        new install   

                                     Brkr Post  Amp Post  GFCI rqt  \
group      Brkr Pre attrStr                                          
Basement_R 7        20A, AFCI, GFCI      False        20         1   
                    20A, AFCI, GFCI      False        20         1   
           0        20A, AFCI, GFCI      False        20         1   
                    20A, AFCI, GFCI      False        20         1   

                                    Box OW int. 1G Box OW ext 1G  ...  \
group      Brkr Pre attrStr                                       ...   
Basement_R 7        20A, AFCI, GFCI          False         False  ...   
                    20A, AFCI, GFCI          False         False  ...   
           0        20A, AFCI, GFCI          False         False  ...   
                    20A, AFCI, GFCI          False         False  ...   

                                    MC conn. SM and/or CO det  Salvage  \
group      Brkr Pre attrStr                                              
Basement_R 7        20A, AFCI, GFCI    False            False    False   
                    20A, AFCI, GFCI    False            False    False   
           0        20A, AFCI, GFCI    False            False    False   
                    20A, AFCI, GFCI    False            False    False   

                                                                     Job  \
group      Brkr Pre attrStr                                                
Basement_R 7        20A, AFCI, GFCI  replace box, connect to existing NM   
                    20A, AFCI, GFCI            None, unless make it 20 A   
           0        20A, AFCI, GFCI                                False   
                    20A, AFCI, GFCI                                False   

                                    Questions is20A isAFCI isGFCI isRemoved  \
group      Brkr Pre attrStr                                                   
Basement_R 7        20A, AFCI, GFCI     False  True   True   True     False   
                    20A, AFCI, GFCI     False  True   True   True     False   
           0        20A, AFCI, GFCI     False  True   True   True     False   
                    20A, AFCI, GFCI     False  True   True   True     False   

                                     is15A  
group      Brkr Pre attrStr                 
Basement_R 7        20A, AFCI, GFCI  False  
                    20A, AFCI, GFCI  False  
           0        20A, AFCI, GFCI  False  
                    20A, AFCI, GFCI  False  

[4 rows x 46 columns]

,,,Fixture ID,Type,Room,Details Pre,New Details Post,Brkr Post,Amp Post,GFCI rqt,Box OW int. 1G,Box OW ext 1G,...,MC conn.,SM and/or CO det,Salvage,Job,Questions,is20A,isAFCI,isGFCI,isRemoved,is15A
group,Brkr Pre,attrStr,,,,,,,,,,,,,,,,,,,,,
BathroomGFCI,13,"20A, AFCI",R12,R,Bathroom,"rcpt, GFCI currently",new ckt,False,20,1,False,False,...,False,False,False,"just rewire to panel, plastic box & Romex good",False,True,True,False,False,False


,,,Fixture ID,Type,Room,Details Pre,New Details Post,Brkr Post,Amp Post,GFCI rqt,Box OW int. 1G,Box OW ext 1G,...,MC conn.,SM and/or CO det,Salvage,Job,Questions,is20A,isAFCI,isGFCI,isRemoved,is15A
group,Brkr Pre,attrStr,,,,,,,,,,,,,,,,,,,,,
FurnaceWH,16,"20A, AFCI, GFCI",R11,R,Basement,"Furnace, Water Heater",new ckt,False,20,1,False,False,...,False,False,False,"None, unless make it 20 A",False,True,True,True,False,False


,,,Fixture ID,Type,Room,Details Pre,New Details Post,Brkr Post,Amp Post,GFCI rqt,Box OW int. 1G,Box OW ext 1G,...,MC conn.,SM and/or CO det,Salvage,Job,Questions,is20A,isAFCI,isGFCI,isRemoved,is15A
group,Brkr Pre,attrStr,,,,,,,,,,,,,,,,,,,,,
Laundry_R,11,"20A, AFCI, GFCI",R8,R,Basement,"R, ironing board on dedicated 20A","GFI'ed, new ckt",False,20,1,False,False,...,False,False,False,"All OK, just need GFCI",False,True,True,True,False,False


,,,Fixture ID,Type,Room,Details Pre,New Details Post,Brkr Post,Amp Post,GFCI rqt,Box OW int. 1G,Box OW ext 1G,...,MC conn.,SM and/or CO det,Salvage,Job,Questions,is20A,isAFCI,isGFCI,isRemoved,is15A
group,Brkr Pre,attrStr,,,,,,,,,,,,,,,,,,,,,
OldElecDryer,"2,4",Removed,R7,R,Basement,Extinct Dryer,Removed. No plug now,False,0,0,False,False,...,False,False,False,remove,Looks like it goes to 2-4 bkr? Reuse breaker s...,False,False,False,True,False


...

,,,Fixture ID,Type,Room,Details Pre,New Details Post,Brkr Post,Amp Post,GFCI rqt,Box OW int. 1G,Box OW ext 1G,...,MC conn.,SM and/or CO det,Salvage,Job,Questions,is20A,isAFCI,isGFCI,isRemoved,is15A
group,Brkr Pre,attrStr,,,,,,,,,,,,,,,,,,,,,
microwave,12,"20A, AFCI",R15,R,Kitchen,microwave,new ckt,False,20,0,False,False,...,False,False,False,rewire at box,False,True,True,False,False,False


Fixture ID Type         Room  \
group         Brkr Pre attrStr                                  
officeDinRm_L 6,8      15A, AFCI        S11    S       Office   
                       15A, AFCI        L6a    L       Office   
              0        15A, AFCI       R24a    S       Office   
              14       15A, AFCI         S8    S  Dining Room   
                       15A, AFCI         L7    L  Dining Room   

                                            Details Pre  \
group         Brkr Pre attrStr                            
officeDinRm_L 6,8      15A, AFCI               light sw   
                       15A, AFCI                  light   
              0        15A, AFCI       behind vis chair   
              14       15A, AFCI  new LED dimmer switch   
                       15A, AFCI          DR chandelier   

                                            New Details Post  Brkr Post  \
group         Brkr Pre attrStr                                            
officeDinRm_L 6,8      15A, AFCI                     new ckt      False   
                       15A, AFCI                     new ckt      False   
              0        15A, AFCI                     install      False   
              14       15A, AFCI  no-blink chand., 2G w/ S16      False   
                       15A, AFCI           no change but box      False   

                                  Amp Post  GFCI rqt Box OW int. 1G  \
group         Brkr Pre attrStr                                        
officeDinRm_L 6,8      15A, AFCI        15         0            1.0   
                       15A, AFCI        15         0          False   
              0        15A, AFCI        20         0            1.0   
              14       15A, AFCI        15         0            1.0   
                       15A, AFCI        15         0          False   

                                 Box OW ext 1G  ... MC conn. SM and/or CO det  \
group         Brkr Pre attrStr                  ...                             
officeDinRm_L 6,8      15A, AFCI         False  ...    False            False   
                       15A, AFCI         False  ...    False            False   
              0        15A, AFCI         False  ...    False            False   
              14       15A, AFCI         False  ...    False            False   
                       15A, AFCI         False  ...    False            False   

                                       Salvage                Job  \
group         Brkr Pre attrStr                                      
officeDinRm_L 6,8      15A, AFCI         False  rewire to breaker   
                       15A, AFCI         False            install   
              0        15A, AFCI         False            install   
              14       15A, AFCI  1 W sw plate     combine w/ S16   
                       15A, AFCI         False              False   

                                                     Questions  is20A isAFCI  \
group         Brkr Pre attrStr                                                 
officeDinRm_L 6,8      15A, AFCI                         False  False   True   
                       15A, AFCI                         False  False   True   
              0        15A, AFCI                         False  False   True   
              14       15A, AFCI  expand to 2G, put S16 in it?  False   True   
                       15A, AFCI                         False  False   True   

                                 isGFCI isRemoved is15A  
group         Brkr Pre attrStr                           
officeDinRm_L 6,8      15A, AFCI  False     False  True  
                       15A, AFCI  False     False  True  
              0        15A, AFCI  False     False  True  
              14       15A, AFCI  False     False  True  
                       15A, AFCI  False     False  True  

[5 rows x 46 columns]

Fixture ID  Type          Room     Details Pre  \
group     Brkr Pre attrStr                                                    
smokedets 0        15A, AFCI        SM1  SMCO    Upper Hall  smoke det + CO   
                   15A, AFCI        SM2    SM  Upper E Bdrm       smoke det   
                   15A, AFCI        SM3    SM  Upper W Bdrm      smoke det    
                   15A, AFCI        SM4  SMCO   Ground Hall  smoke det + CO   
                   15A, AFCI        SM5    SM        Office        smok det   
                   15A, AFCI        SM6  SMCO      Basement  smoke det + CO   

                                     New Details Post  Brkr Post  Amp Post  \
group     Brkr Pre attrStr                                                   
smokedets 0        15A, AFCI  new inst BRK SC9120B-3       False        15   
                   15A, AFCI       new inst BRK 3102B      False        15   
                   15A, AFCI       new inst BRK 3102B      False        15   
                   15A, AFCI  new inst BRK SC9120B-3       False        15   
                   15A, AFCI       new inst BRK 3102B      False        15   
                   15A, AFCI  new inst BRK SC9120B-3       False        15   

                              GFCI rqt Box OW int. 1G Box OW ext 1G  ...  \
group     Brkr Pre attrStr                                           ...   
smokedets 0        15A, AFCI         0          False         False  ...   
                   15A, AFCI         0          False         False  ...   
                   15A, AFCI         0          False         False  ...   
                   15A, AFCI         0          False         False  ...   
                   15A, AFCI         0          False         False  ...   
                   15A, AFCI         0          False         False  ...   

                             MC conn. SM and/or CO det  Salvage    Job  \
group     Brkr Pre attrStr                                               
smokedets 0        15A, AFCI    False              1.0    False  False   
                   15A, AFCI    False              1.0    False  False   
                   15A, AFCI    False              1.0    False  False   
                   15A, AFCI    False              1.0    False  False   
                   15A, AFCI    False              1.0    False  False   
                   15A, AFCI    False              1.0    False  False   

                             Questions  is20A isAFCI isGFCI isRemoved is15A  
group     Brkr Pre attrStr                                                   
smokedets 0        15A, AFCI     False  False   True  False     False  True  
                   15A, AFCI     False  False   True  False     False  True  
                   15A, AFCI     False  False   True  False     False  True  
                   15A, AFCI     False  False   True  False     False  True  
                   15A, AFCI     False  False   True  False     False  True  
                   15A, AFCI     False  False   True  False     False  True  

[6 rows x 46 columns]

Fixture ID Type          Room  \
group                  Brkr Pre attrStr                                   
upperBdrmsUpperStair_L 6,8      15A, AFCI        S10    S   Ground Hall   
                                15A, AFCI        L10    L   Ground Hall   
                                15A, AFCI        L10    L    Upper Hall   
                                15A, AFCI        S13    S    Upper Hall   
                                15A, AFCI        S15    S  Upper W Bdrm   
                                15A, AFCI        L11    L  Upper W Bdrm   
                                15A, AFCI        L12    L  Upper E Bdrm   
                                15A, AFCI        S12    S  Upper E Bdrm   

                                                          Details Pre  \
group                  Brkr Pre attrStr                                 
upperBdrmsUpperStair_L 6,8      15A, AFCI  sw to upper/lower H lights   
                                15A, AFCI                  Hall light   
                                15A, AFCI                       light   
                                15A, AFCI  sw to upper/lower H lights   
                                15A, AFCI                   sw to L11   
                                15A, AFCI                       light   
                                15A, AFCI                       light   
                                15A, AFCI                  SW, on L12   

                                          New Details Post  Brkr Post  \
group                  Brkr Pre attrStr                                 
upperBdrmsUpperStair_L 6,8      15A, AFCI          new ckt      False   
                                15A, AFCI          new ckt      False   
                                15A, AFCI          new ckt      False   
                                15A, AFCI          new ckt      False   
                                15A, AFCI          new ckt      False   
                                15A, AFCI          new ckt      False   
                                15A, AFCI          new ckt      False   
                                15A, AFCI          new ckt      False   

                                           Amp Post  GFCI rqt Box OW int. 1G  \
group                  Brkr Pre attrStr                                        
upperBdrmsUpperStair_L 6,8      15A, AFCI        15         0            1.0   
                                15A, AFCI        15         0            1.0   
                                15A, AFCI        15         0          False   
                                15A, AFCI        15         0            1.0   
                                15A, AFCI        15         0            1.0   
                                15A, AFCI        15         0          False   
                                15A, AFCI        15         0          False   
                                15A, AFCI        15         0            1.0   

                                          Box OW ext 1G  ... MC conn.  \
group                  Brkr Pre attrStr                  ...            
upperBdrmsUpperStair_L 6,8      15A, AFCI         False  ...    False   
                                15A, AFCI         False  ...    False   
                                15A, AFCI         False  ...    False   
                                15A, AFCI         False  ...    False   
                                15A, AFCI         False  ...    False   
                                15A, AFCI         False  ...    False   
                                15A, AFCI           1.0  ...    False   
                                15A, AFCI         False  ...    False   

                                          SM and/or CO det  Salvage  \
group                  Brkr Pre attrStr                               
upperBdrmsUpperStair_L 6,8      15A, AFCI            False    False   
                                15A, AFCI            False    False   
             

Fixture ID Type          Room  \
group        Brkr Pre attrStr                                   
upperFloor_R 14       20A, AFCI        R30    R  Upper W Bdrm   
             0        20A, AFCI       R30a    R  Upper W Bdrm   
                      20A, AFCI        R31    R    Upper Hall   
             6,8      20A, AFCI        R29    R  Upper W Bdrm   
             0        20A, AFCI       R29a    R  Upper W Bdrm   
                      20A, AFCI       R29b    R  Upper W Bdrm   
                      20A, AFCI       R29c    R  Upper W Bdrm   
                      20A, AFCI        R26    R  Upper E Bdrm   
             6,8      20A, AFCI        R27    R  Upper E Bdrm   
             0        20A, AFCI       R27a    R  Upper E Bdrm   
                      20A, AFCI       R27b    R  Upper E Bdrm   
                      20A, AFCI       R27c    R  Upper E Bdrm   
             14       20A, AFCI        R28    R  Upper E Bdrm   

                                             Details Pre New Details Post  \
group        Brkr Pre attrStr                                               
upperFloor_R 14       20A, AFCI              N wall rcpt          new ckt   
             0        20A, AFCI            W wall middle      new install   
                      20A, AFCI  middle, optional recpt.      new install   
             6,8      20A, AFCI              W wall rcpt          new ckt   
             0        20A, AFCI           S wall, W side      new install   
                      20A, AFCI           W wall, S side      new install   
                      20A, AFCI           W wall, N side      new install   
                      20A, AFCI       N wall near closet      new install   
             6,8      20A, AFCI            N wall E side          new ckt   
             0        20A, AFCI           E wall, N side      new install   
                      20A, AFCI           E wall, S side      new install   
                      20A, AFCI          S wall , E side      new install   
             14       20A, AFCI           S wall, W side          new ckt   

                                 Brkr Post  Amp Post  GFCI rqt Box OW int. 1G  \
group        Brkr Pre attrStr                                                   
upperFloor_R 14       20A, AFCI      False        20         0          False   
             0        20A, AFCI      False        20         0          False   
                      20A, AFCI      False        20         0            1.0   
             6,8      20A, AFCI      False        20         0          False   
             0        20A, AFCI      False        20         0          False   
                      20A, AFCI      False        20         0          False   
                      20A, AFCI      False        20         0          False   
                      20A, AFCI      False        20         0          False   
             6,8      20A, AFCI      False        20         0          False   
             0        20A, AFCI      False        20         0          False   
                      20A, AFCI      False        20         0          False   
                      20A, AFCI      False        20         0          False   
             14       20A, AFCI      False        20         0          False   

                                Box OW ext 1G  ... MC conn. SM and/or CO det  \
group        Brkr Pre attrStr                  ...                             
upperFloor_R 14       20A, AFCI           1.0  ...    False            False   
             0        20A, AFCI           1.0  ...    False            False   
                      20A, AFCI         False  ...    False            False   
             6,8      20A, AFCI           1.0  ...    False            False   
             0        20A, AFCI           1.0  ...    False            False   
                      20A, AFCI           1.0  ...    False            False   
                      20A, AFCI           1.

### Number of fixtures per group

In [22]:
fixture_counts = pd.DataFrame(index=df_fg.group.unique(), columns=df_fg.Type.unique())
for gt_ix, dfg in df_fg.groupby(['group','Type']):
    fixture_counts.loc[gt_ix] = len(dfg)
fixture_counts.fillna(0)

,S,R,L,F,SMCO,SM
garagBsmnt_L,5,0,11,0,0,0
garagExtSwall_GFCI,0,6,0,0,0,0
clothesWasher,0,1,0,0,0,0
OldElecDryer,0,1,0,0,0,0
Laundry_R,0,1,0,0,0,0
Basement_R,0,4,0,0,0,0
FurnaceWH,0,1,0,0,0,0
upperBdrmsUpperStair_L,4,0,4,0,0,0
groundFloorR_extNS_LRF,1,14,3,3,0,0
kitchenBath_L,4,0,8,2,0,0


In [23]:
# tally up breaker attributes
fixture_counts = pd.DataFrame(df_fg.groupby(['attrStr','group']).count().iloc[:, 0])
breaker_attribute_counts = pd.DataFrame(fixture_counts.reset_index().groupby(['attrStr']).count().iloc[:, 0]).sort_index()
display(breaker_attribute_counts)
display(fixture_counts)

,group
attrStr,
"15A, AFCI",5
"20A, AFCI",10
"20A, AFCI, GFCI",4
Removed,1


Fixture ID
attrStr         group                             
15A, AFCI       garagBsmnt_L                    16
                kitchenBath_L                   14
                officeDinRm_L                    5
                smokedets                        6
                upperBdrmsUpperStair_L           8
20A, AFCI       BathroomGFCI                     1
                Oven                             1
                dishwasher                       1
                fridge                           1
                garagExtSwall_GFCI               6
                groundFloorR_extNS_LRF          21
                kitchenGFCI_N                    2
                kitchenGFCI_S                    2
                microwave                        1
                upperFloor_R                    13
20A, AFCI, GFCI Basement_R                       4
                FurnaceWH                        1
                Laundry_R                        1
                clothesWasher                    1
Removed         OldElecDryer                     1

#### Number of dedicated circuits

In [24]:
nTypeF = df_fg[df_fg['Type'].str.startswith('F')].shape[0]
nTypeFdedicated = 2 # F1 and F4: fan and garbage disposal
nTypeFdedicatedInspected=1 # just Fan: F1
nDedicatedOther =  5 # smokedets, bathroomGFCI, exterior S wall GFCI, Furnace and waterheater, Laundry, washer_gas dryer.
print(f'Number of Dedicated Fixtures to be inspected {nTypeFdedicatedInspected + nDedicatedOther}')

Number of Dedicated Fixtures to be inspected 6


In [25]:
# Number of each device type
df_fg.groupby('Type').count().iloc[:,0]

# Number of inspectable fixtures

nLights = df_fg.query('Type=="L" and Room not in () ')

### Cable lengths

In [26]:
df_fg.Room.unique()

dist_from_box = pd.DataFrame(index=['Garage', 'Basement', 'Ground Hall', 'Basement Stairs', 'Office',
       'Bathroom', 'Kitchen', 'E Exterior Wall', 'Dining Room',
       'N Exterior Wall', 'Living Room', 'S Exterior Wall', 'Upper Hall',
       'Upper W Bdrm', 'Upper E Bdrm'], columns=['feet'])
dist_from_box

,feet
Garage,NaN
Basement,NaN
Ground Hall,NaN
Basement Stairs,NaN
Office,NaN
Bathroom,NaN
Kitchen,NaN
E Exterior Wall,NaN
Dining Room,NaN
N Exterior Wall,NaN


In [27]:
# On my OneNote drawing:
# X is NS (N down)
# Y is WE (W left)
# Z is up
# origin is basement floor, NE corner
# dims in feet

ground_floor_height = 95 /12
ceiling_dist = (91+12) / 12
upper_floor_height = ground_floor_height + ceiling_dist
socket_dist = 16 /12
switch_dist = 48 / 12

main_box_center = np.array([0, 11.1, 0]) # x, y, z

# location furthest from box
room_max_xyz = {'Garage':[(22+3.3+19)/12, 13.8, 0],
              'Basement':[369/12, 20.8, 0],
              'Ground Hall':[369/12, 10.5+6, ground_floor_height],
              'Basement Stairs':[369/2, 12, ground_floor_height],
              'Office':[13.6+4, 11.5, ground_floor_height],
              'Bathroom':[3, 3, upper_floor_height],
              'Kitchen':[3, 3, upper_floor_height],
              'E Exterior Wall':[3, 3, upper_floor_height],
              'Dining Room':[10, 20, upper_floor_height],
              'N Exterior Wall':[20.8, 30.8, upper_floor_height],
              'Living Room':[20.8, 30.8, ground_floor_height],
              'S Exterior Wall':[20.8, 30.8, ground_floor_height],
              'Upper Hall':[20.8/2, 30.8/2, upper_floor_height+ceiling_dist],
              'Upper W Bdrm':[20.8, 30.8, upper_floor_height+ceiling_dist],
              'Upper E Bdrm':[20.8*2/3, 10.5, upper_floor_height+ceiling_dist]}

wirelen = 0
for breakerNm, dfbk in df_fg.groupby('group'):
    for room, dfr in dfbk.groupby('Room'):
        roomdist =  np.sum(np.absolute(np.array(room_max_xyz[room]) - main_box_center))
        wirelen += roomdist


                            

In [28]:
wirelen/100


17.27066666666666